In [4]:
import numpy as np
from scipy import linalg
import os


# 类及函数封装



In [5]:
def solve_sym(xtx, xty):
    L = linalg.cholesky(xtx)
    return linalg.lapack.dpotrs(L, xty)[0]


我们使用递推公式获得矩阵$Z$

$$
\begin{array}{cc}
 & p=1 \\
\begin{array}{c}0\\1\end{array}&
\left[\begin{array}{cc}
0\\
1
\end{array}\right]
\end{array}
\Rightarrow
\begin{array}{cc}
 & p=2 \\
\begin{array}{c}0\\2\\1\\3\end{array}&
\left[\begin{array}{cc}
0&0\\
0&1\\
1&0\\
1&1
\end{array}\right]
\end{array}
\Rightarrow
\begin{array}{cc}
 & p=3 \\
\begin{array}{c}0\\4\\2\\6\\1\\5\\3\\7\end{array}&
\left[\begin{array}{ccc}
0&0&0\\
0&0&1\\
0&1&0\\
0&1&1\\
1&0&0\\
1&0&1\\
1&1&0\\
1&1&1
\end{array}\right]
\end{array}\quad \cdots \tag{2}
$$

- 上面$(2)$式中矩阵左侧的数字代表矩阵行向量的十进制数值.
- 从$(2)$式中可以看出，给矩阵左侧添加一列$0$得到的向量，其十进制值为原向量的二倍，给矩阵的左侧添加一列$1$得到的向量，其十进制值为原向量的二倍加一.
- 给$2^p*p$维$Z$矩阵左侧分别添加一列$0$和$1$，并按行拼接，即可得到$2^{(p+1)}*(p+1)$维$Z$矩阵.
- turnbits_rec函数即可实现此功能。



In [6]:
def turnbits_rec(p):
    if(p==1):
        return np.array([[True, False],[True, True]])
    else:
        tmp1 = np.c_[ turnbits_rec(p - 1),
                     np.array([False]*(2**(p - 1))).reshape((2**(p - 1), 1))] # Create False and then reshape.
        tmp2 = np.c_[ turnbits_rec(p - 1),
                     np.array([True]*(2**(p - 1))).reshape((2**(p - 1), 1))] # Create True and then reshape.
        return np.r_[tmp1, tmp2]

In [7]:
class BestSubsetRegress(object):
    """BestSubsetRegression

    A best subset regression based on Cp, AIC and cross validation.

    Attributes:
        x: A ndarray data of dependent variable.
        y: A ndarray data of independent variable.
        intercept: A boolean indicating if intercept is included in data.
        isCp: A boolean indicating if Cp is applied.
        isAIC: A boolean indicating if AIC is applied.
        isCV: A boolean indicating if cross validation is applied.
    """
    def __init__(self, x=0,  y=0, intercept=True, isCp=True, isAIC=True, isCV=True):
        self.n, self.p = x.shape
        if intercept:
            self.x = np.c_[np.ones((self.n, 1)), x] # If intercept, then generize a column which is all 1
        else:
            self.x = x

        self.ind_var = turnbits_rec(self.p) # Generate the matrix using the pre-defined function
        self.y = y
        self.xtx = np.dot(self.x.T, self.x)
        self.xty = np.dot(self.x.T, self.y)
        self.b = [] # Regression Coefficients

        # Parameters
        self.isCp = isCp
        self.isAIC = isAIC
        self.isCV = isCV
        self.intercept = intercept

    # Calculate Regression Coefficients
    def regress(self):
        self.b = [solve_sym(self.xtx[ind][:, ind], self.xty[ind]) # "ind" is the index of the variables.
                  for ind in self.ind_var]
        return self.b

    # Calculate CV statistics
    def Cp_AIC(self):
        self.b = BestSubsetRegress.regress(self)
        mse = [np.sum(np.dot(self.xtx[ind][:, ind], b_)*b_)
               for ind, b_ in zip(self.ind_var, self.b)]
        rss = np.dot(self.y, self.y) - mse
        d = np.sum(self.ind_var, axis=1)
        self.Cp = rss + 2*d*rss[-1]/(self.n - self.p - 1)
        self.AIC = self.n*np.log(rss) + 2*d

    # Performing Cross Validation
    def cvreg(self):
        K = 10
        indices = np.array_split(np.random.permutation(np.arange(0, self.n)), K) # Split the datasets into K folds.

        def cvk(ind, index): # "ind" is the index of the variables. "index" is the index of the index of the Kth fold.
            txx = self.xtx[ind][:, ind] - np.dot((self.x[index][:, ind]).T,
                                                 self.x[index][:, ind])
            txy = self.xty[ind] - np.dot((self.x[index][:, ind]).T,
                                         self.y[index])
            tcoe = solve_sym(txx, txy)
            return np.sum(
                (self.y[index] - np.dot(self.x[index][:, ind], tcoe))**2)
        self.cverr = np.sum(np.array(
            [[cvk(ind, index) for index in indices]
             for ind in self.ind_var]), axis = 1)/self.n


    def bestsubsetregress(self):
        self.names = np.array(['x' + str(i) for i in range(1, self.p+1)])

        if self.isCp or self.isAIC:
            BestSubsetRegress.Cp_AIC(self)

        if self.isCp:
            min_id_Cp = np.argmin(self.Cp)
            if self.intercept:
                sub_names_Cp = np.insert(self.names[self.ind_var[min_id_Cp][1:]], 0, 'mu')
            else:
                sub_names_Cp = self.names[self.ind_var[min_id_Cp][1:]]
            sub_beta_Cp = self.b[min_id_Cp]
            print('The best subset and coeffecients under Cp:\n')
            print(dict(zip(sub_names_Cp, sub_beta_Cp)), '\n')

        if self.isAIC:
            min_id_AIC = np.argmin(self.AIC)
            if self.intercept:
                sub_names_AIC = np.insert(self.names[self.ind_var[min_id_AIC][1:]], 0, 'mu')
            else:
                sub_names_AIC = self.names[self.ind_var[min_id_AIC][1:]]
            sub_beta_AIC = self.b[min_id_AIC]
            print('The best subset and coeffecients under AIC:\n')
            print(dict(zip(sub_names_AIC, sub_beta_AIC)), '\n')

        if self.isCV:
            BestSubsetRegress.cvreg(self)
            min_id_CV = np.argmin(self.cverr)
            if self.intercept:
                sub_names_CV = np.insert(self.names[self.ind_var[min_id_CV][1:]], 0, 'mu')
            else:
                sub_names_CV = self.names[self.ind_var[min_id_CV][1:]]
            sub_beta_CV = self.b[min_id_CV]
            print('The best subset and coeffecients under CV:\n')
            print(dict(zip(sub_names_CV, sub_beta_CV)), '\n')


# 结果



In [8]:
#os.chdir()
x = np.loadtxt("./DataForRegression/x.txt", delimiter=",")
y = np.loadtxt("./DataForRegression/y.txt", delimiter=",")
regress = BestSubsetRegress(x, y)
regress.bestsubsetregress()

The best subset and coeffecients under Cp:

{'mu': 0.49472926218273405, 'x1': 0.5439978569443544, 'x2': 0.58821270309517, 'x3': -0.01644484649754566, 'x4': 0.10122333372346587, 'x5': 0.714903976347165} 

The best subset and coeffecients under AIC:

{'mu': 0.49472926218273405, 'x1': 0.5439978569443544, 'x2': 0.58821270309517, 'x3': -0.01644484649754566, 'x4': 0.10122333372346587, 'x5': 0.714903976347165} 

The best subset and coeffecients under CV:

{'mu': -0.77715664158, 'x1': 0.5258518819809425, 'x2': 0.6617699115944506, 'x5': 0.6656665628572002} 




# 运行指标

所有模型的 Cp 统计量



In [9]:
regress.Cp

array([128.89625931,  60.87198491, 105.85631228,  54.6779773 ,
       126.19572643,  61.8480482 , 106.78341241,  55.01093637,
       125.73909996,  58.58481473, 106.81264898,  55.21253879,
       125.06011836,  59.16845908, 107.71877639,  55.18056728,
        88.8640649 ,  56.61309532,  74.21000979,  50.48283814,
        88.46726558,  57.58805268,  75.16117447,  50.8384296 ,
        83.11919498,  53.20060014,  74.44826612,  50.4884743 ,
        84.03757715,  53.64369376,  75.27236112,  50.30828436,
        91.34658525,  61.19438516,  76.85530546,  54.83786119,
        91.03708423,  62.16693579,  77.79760015,  55.20521981,
        88.01061732,  58.8012657 ,  77.71425717,  55.34666926,
        88.8066033 ,  59.40963385,  78.61296632,  55.35018109,
        83.26614882,  57.4478506 ,  70.13680559,  51.39206599,
        83.23329324,  58.42428262,  71.04891477,  51.73711287,
        78.37878135,  53.98555937,  70.503571  ,  51.3659207 ,
        79.33771078,  54.39454394,  71.26396274,  51.15


所有模型的AIC



In [10]:
regress.AIC

array([472.5845164 , 399.38092999, 454.41177355, 388.78850356,
       471.75372119, 401.37675265, 456.36368106, 389.57051796,
       471.39655046, 395.84924193, 456.39098595, 389.95248042,
       472.08883328, 397.15835211, 458.32328717, 390.02250977,
       437.08919065, 392.35003799, 419.85384941, 380.56952925,
       437.54177823, 394.34345438, 421.81650379, 381.26307028,
       431.27869393, 386.07147685, 420.84101526, 380.52141645,
       433.20581894, 387.01178031, 422.62830233, 380.02433362,
       439.82118744, 400.29447009, 423.38874181, 389.24139995,
       440.41324824, 402.28439671, 425.34108834, 390.07005054,
       437.02251264, 396.5115827 , 425.23160699, 390.34237695,
       438.8140768 , 397.85506495, 427.12654375, 390.44936244,
       431.45630523, 394.08974713, 414.72282679, 382.4249215 ,
       432.22720391, 396.08581801, 416.62538408, 383.09420225,
       426.10112589, 387.68962164, 415.822469  , 382.30598189,
       428.07549835, 388.55756031, 417.49932718, 381.73